raga:https://docs.ragas.io/en/latest/getstarted/

TrueLence:https://www.trulens.org/

ares:https://github.com/stanford-futuredata/ARES?tab=readme-ov-file#section3

In [1]:
from langchain.document_loaders import TextLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
pip install ragas

  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
Using cached xxhash-3.5.0-cp310-cp310-win_amd64.whl (30 kB)

  Attempting uninstall: fsspec

    Found existing installation: fsspec 2025.5.1

    Uninstalling fsspec-2025.5.1:

      Successfully unins

In [2]:
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
loader=TextLoader(r"D:\Archieve\Langchain_components\7.RAG_Evaluation\japan.txt")

In [4]:
documents=loader.load()

In [5]:
documents

[Document(metadata={'source': 'D:\\Archieve\\Langchain_components\\7.RAG_Evaluation\\japan.txt'}, page_content='Industrial revival hope for Japan\n\nJapanese industry is growing faster than expected, boosting hopes that the country\'s retreat back into recession is over.\n\nIndustrial output rose 2.1% - adjusted for the time of year - in January from a month earlier. At the same time, retail sales picked up faster than at any time since 1997. The news sent Tokyo shares to an eight-month high, as investors hoped for a recovery from the three quarters of contraction seen from April 2004 on. The Nikkei 225 index ended the day up 0.7% at 11,740.60 points, with the yen strengthening 0.7% against the dollar to 104.53 yen. Weaker exports, normally the engine for Japan\'s economy in the face of weak domestic demand, had helped trigger a 0.1% contraction in the final three months of last year after two previous quarters of shrinking GDP. Only an exceptionally strong performance in the early mon

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
splits = text_splitter.split_documents(documents)

In [7]:
len(splits)

29

In [10]:
from ragas.llms import LangchainLLMWrapper
llm = ChatGroq(model="llama3-70b-8192")

d:\Archieve\Langchain_components\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

C:\Users\Suman Vemula\AppData\Local\Temp\ipykernel_13704\2127729888.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [12]:
vectorstore = FAISS.from_documents(documents=splits,embedding=embedding_model)

In [13]:
retriever=vectorstore.as_retriever()

In [14]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [15]:
prompt = ChatPromptTemplate.from_template(template)

In [16]:
rag_chain=(
    {"context":retriever, "question":RunnablePassthrough()}
          | prompt
          | llm
          | StrOutputParser()
)

In [17]:
questions = [
    "What is the current status of Japan's industrial production?",
    "How have retail sales in Japan performed recently?",
    "What did the Bank of Japan say about the economic outlook?",
    "What has been the performance of the Nikkei 225 index?",
    "How has the Japanese yen moved in recent times?",
    "What are some challenges Japan's economy is still facing?"
]

In [18]:
ground_truths = ["Japan's industrial production grew by 2.3% in February 2025, indicating a moderate recovery after previous declines.",
 'Retail sales in Japan rose by 3.9% year-on-year in January 2025, marking the 36th consecutive month of growth.',
 'The Bank of Japan stated that there is a low risk of serious recession, but warned of uncertainties and revised growth forecasts downward.',
 'The Nikkei 225 index has performed strongly, reaching around 11,740 points and is projected to climb to 45,400 by the end of 2025.',
 'The Japanese yen has strengthened to around 104.53 against the US dollar, reflecting renewed investor optimism.',
 "Japan's economy continues to face challenges such as weak exports, trade tensions, and an aging population despite recent growth."]

In [19]:
rag_chain.invoke(questions[1])

'Retail sales in Japan have shown consistent growth, rising by 3.9% year-on-year in January, marking the 36th consecutive month of expansion. Additionally, retail sales picked up faster than at any time since 1997.'

In [20]:
answer=[]
context=[]

In [21]:
for query in questions:
  answer.append(rag_chain.invoke(query))
  context.append([docs.page_content  for docs in retriever.invoke(query)])

In [22]:
answer

["As of mid-2025, Japan's industrial production experienced a 1.1% month-over-month decline. However, earlier in 2025, industrial production had increased by 2.3% month-over-month in February, indicating a rebound from previous contractions.",
 'Retail sales in Japan have shown consistent growth, rising by 3.9% year-on-year in January, marking the 36th consecutive month of expansion. Additionally, retail sales picked up faster than at any time since 1997.',
 'The Bank of Japan remains cautiously optimistic about the economic outlook, noting that while certain economic indicators are encouraging, vigilance is necessary to sustain growth. However, it also revised its growth projection for the fiscal year ending March 2026 from 1.1% to 0.5% due to uncertainties stemming from global trade tensions.',
 'The Nikkei 225 index has shown strength, reaching levels around 11,740.60 points and is projected to reach 45,400 by the end of 2025. It has also shown a positive performance, ending the day

In [23]:
context

[["As of early 2025, Japan's economy exhibits signs of moderate recovery, with industrial production increasing by 2.3% month-over-month in February, slightly below initial estimates but indicating a rebound from previous contractions. Retail sales",
  "Industrial revival hope for Japan\n\nJapanese industry is growing faster than expected, boosting hopes that the country's retreat back into recession is over.",
  "As of mid-2025, Japan's economy is experiencing a nuanced recovery characterized by moderate growth and persistent challenges. In the first quarter, real GDP contracted by 0.1%, influenced by a 1.1% month-over-month decline in industrial production",
  "Japan's last four year GDP:\n\nYear\tGDP\n2022\t$4,256.41B\n2021\t$5,034.62B\n2020\t$5,055.59B\n2019\t$5,117.99B"],
 ["As of early 2025, Japan's economy exhibits signs of moderate recovery, with industrial production increasing by 2.3% month-over-month in February, slightly below initial estimates but indicating a rebound from

In [24]:
contexts = [" ".join(ref) if isinstance(ref, list) else ref for ref in context]

In [25]:
contexts[0]

"As of early 2025, Japan's economy exhibits signs of moderate recovery, with industrial production increasing by 2.3% month-over-month in February, slightly below initial estimates but indicating a rebound from previous contractions. Retail sales Industrial revival hope for Japan\n\nJapanese industry is growing faster than expected, boosting hopes that the country's retreat back into recession is over. As of mid-2025, Japan's economy is experiencing a nuanced recovery characterized by moderate growth and persistent challenges. In the first quarter, real GDP contracted by 0.1%, influenced by a 1.1% month-over-month decline in industrial production Japan's last four year GDP:\n\nYear\tGDP\n2022\t$4,256.41B\n2021\t$5,034.62B\n2020\t$5,055.59B\n2019\t$5,117.99B"

In [26]:
ground_truths[0]

"Japan's industrial production grew by 2.3% in February 2025, indicating a moderate recovery after previous declines."

In [27]:
# To dict
data = {
    "question": questions,
    "answer": answer,
    "contexts": context,
    "reference": ground_truths
}

In [28]:
data

{'question': ["What is the current status of Japan's industrial production?",
  'How have retail sales in Japan performed recently?',
  'What did the Bank of Japan say about the economic outlook?',
  'What has been the performance of the Nikkei 225 index?',
  'How has the Japanese yen moved in recent times?',
  "What are some challenges Japan's economy is still facing?"],
 'answer': ["As of mid-2025, Japan's industrial production experienced a 1.1% month-over-month decline. However, earlier in 2025, industrial production had increased by 2.3% month-over-month in February, indicating a rebound from previous contractions.",
  'Retail sales in Japan have shown consistent growth, rising by 3.9% year-on-year in January, marking the 36th consecutive month of expansion. Additionally, retail sales picked up faster than at any time since 1997.',
  'The Bank of Japan remains cautiously optimistic about the economic outlook, noting that while certain economic indicators are encouraging, vigilance

In [29]:
import pandas as pd
pd.DataFrame(data)

,question,answer,contexts,reference
0,What is the current status of Japan's industri...,"As of mid-2025, Japan's industrial production ...","[As of early 2025, Japan's economy exhibits si...",Japan's industrial production grew by 2.3% in ...
1,How have retail sales in Japan performed recen...,Retail sales in Japan have shown consistent gr...,"[As of early 2025, Japan's economy exhibits si...",Retail sales in Japan rose by 3.9% year-on-yea...
2,What did the Bank of Japan say about the econo...,The Bank of Japan remains cautiously optimisti...,[population. The Bank of Japan remains cautiou...,The Bank of Japan stated that there is a low r...
3,What has been the performance of the Nikkei 22...,"The Nikkei 225 index has shown strength, reach...","[In the financial markets, the Nikkei 225 inde...","The Nikkei 225 index has performed strongly, r..."
4,How has the Japanese yen moved in recent times?,"The Japanese yen has experienced fluctuations,...",[return of foreign investors. The Japanese yen...,The Japanese yen has strengthened to around 10...
5,What are some challenges Japan's economy is st...,Japan's economy is still facing challenges inc...,"[As of mid-2025, Japan's economy is experienci...",Japan's economy continues to face challenges s...


In [30]:
from datasets import Dataset
dataset=Dataset.from_dict(data)

In [31]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'reference'],
    num_rows: 6
})

In [32]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [33]:
result=evaluate(
    dataset=dataset,
    metrics=[context_precision,context_recall,faithfulness,answer_relevancy]
)

Evaluating: 100%|██████████| 24/24 [00:20<00:00,  1.15it/s]


In [34]:
result

{'context_precision': 0.7037, 'context_recall': 0.7778, 'faithfulness': 1.0000, 'answer_relevancy': 0.6250}

In [35]:
df=result.to_pandas()

In [36]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,What is the current status of Japan's industri...,"[As of early 2025, Japan's economy exhibits si...","As of mid-2025, Japan's industrial production ...",Japan's industrial production grew by 2.3% in ...,1.000000,1.000000,1.0,0.903890
1,How have retail sales in Japan performed recen...,"[As of early 2025, Japan's economy exhibits si...",Retail sales in Japan have shown consistent gr...,Retail sales in Japan rose by 3.9% year-on-yea...,0.500000,1.000000,1.0,0.959404
2,What did the Bank of Japan say about the econo...,[population. The Bank of Japan remains cautiou...,The Bank of Japan remains cautiously optimisti...,The Bank of Japan stated that there is a low r...,0.500000,0.666667,1.0,0.000000
3,What has been the performance of the Nikkei 22...,"[In the financial markets, the Nikkei 225 inde...","The Nikkei 225 index has shown strength, reach...","The Nikkei 225 index has performed strongly, r...",0.805556,1.000000,1.0,0.913195
4,How has the Japanese yen moved in recent times?,[return of foreign investors. The Japanese yen...,"The Japanese yen has experienced fluctuations,...",The Japanese yen has strengthened to around 10...,1.000000,1.000000,1.0,0.000000
5,What are some challenges Japan's economy is st...,"[As of mid-2025, Japan's economy is experienci...",Japan's economy is still facing challenges inc...,Japan's economy continues to face challenges s...,0.416667,0.000000,1.0,0.973773


In [37]:
df.iloc[0,:]

user_input            What is the current status of Japan's industri...
retrieved_contexts    [As of early 2025, Japan's economy exhibits si...
response              As of mid-2025, Japan's industrial production ...
reference             Japan's industrial production grew by 2.3% in ...
context_precision                                                   1.0
context_recall                                                      1.0
faithfulness                                                        1.0
answer_relevancy                                                0.90389
Name: 0, dtype: object

In [38]:
#data
#this is my actual query
df.iloc[0,:].user_input	

"What is the current status of Japan's industrial production?"

In [39]:
#predicted
#generation of the model
df.iloc[0,:].retrieved_contexts	

["As of early 2025, Japan's economy exhibits signs of moderate recovery, with industrial production increasing by 2.3% month-over-month in February, slightly below initial estimates but indicating a rebound from previous contractions. Retail sales",
 "Industrial revival hope for Japan\n\nJapanese industry is growing faster than expected, boosting hopes that the country's retreat back into recession is over.",
 "As of mid-2025, Japan's economy is experiencing a nuanced recovery characterized by moderate growth and persistent challenges. In the first quarter, real GDP contracted by 0.1%, influenced by a 1.1% month-over-month decline in industrial production",
 "Japan's last four year GDP:\n\nYear\tGDP\n2022\t$4,256.41B\n2021\t$5,034.62B\n2020\t$5,055.59B\n2019\t$5,117.99B"]

In [40]:
#actual
#actual answer(manually we have written this answer)(this could be llm generated)
df.iloc[0,:].response

"As of mid-2025, Japan's industrial production experienced a 1.1% month-over-month decline. However, earlier in 2025, industrial production had increased by 2.3% month-over-month in February, indicating a rebound from previous contractions."

In [41]:
#average value
#retrieval result(context)
df.iloc[0,:].reference	

"Japan's industrial production grew by 2.3% in February 2025, indicating a moderate recovery after previous declines."

In [42]:
#rouge summrization
#bleu translation

In [43]:
from datasets import Dataset
from ragas.metrics import answer_similarity
from ragas import evaluate


data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_similarity])
score.to_pandas()

Evaluating: 100%|██████████| 2/2 [00:00<00:00,  2.18it/s]


,user_input,response,reference,semantic_similarity
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",0.996380
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,0.924222


In [44]:
from datasets import Dataset
from ragas.metrics import faithfulness, answer_correctness
from ragas import evaluate

data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}
dataset = Dataset.from_dict(data_samples)
score = evaluate(dataset,metrics=[answer_correctness])
score.to_pandas()

Evaluating: 100%|██████████| 2/2 [00:05<00:00,  2.74s/it]


,user_input,response,reference,answer_correctness
0,When was the first super bowl?,"The first superbowl was held on Jan 15, 1967","The first superbowl was held on January 15, 1967",0.999104
1,Who won the most super bowls?,The most super bowls have been won by The New ...,The New England Patriots have won the Super Bo...,0.981055
